<a href="https://colab.research.google.com/github/ArmstrongA/fbsuccess/blob/main/optimized_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import os
import time

import warnings
warnings.filterwarnings('ignore')

In [7]:
# Full stats for fbref.com seem to start in 2017
epl = ["https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2021-2022/schedule/2021-2022-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2019-2020/schedule/2019-2020-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2018-2019/schedule/2018-2019-Premier-League-Scores-and-Fixtures",
    "https://fbref.com/en/comps/9/2017-2018/schedule/2017-2018-Premier-League-Scores-and-Fixtures"]

epl_current = ["https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"]

ligue1 = ["https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2021-2022/schedule/2021-2022-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2020-2021/schedule/2020-2021-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2019-2020/schedule/2019-2020-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2018-2019/schedule/2018-2019-Ligue-1-Scores-and-Fixtures",
    "https://fbref.com/en/comps/13/2017-2018/schedule/2017-2018-Ligue-1-Scores-and-Fixtures"]


ligue1_current = ["https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures"]

bundesliga = ["https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2021-2022/schedule/2021-2022-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2020-2021/schedule/2020-2021-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2019-2020/schedule/2019-2020-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2018-2019/schedule/2018-2019-Bundesliga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/20/2017-2018/schedule/2017-2018-Bundesliga-Scores-and-Fixtures"]

bundesliga_current = ["https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures"]

seriea = ["https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2021-2022/schedule/2021-2022-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2020-2021/schedule/2020-2021-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2019-2020/schedule/2019-2020-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2018-2019/schedule/2018-2019-Serie-A-Scores-and-Fixtures",
    "https://fbref.com/en/comps/11/2017-2018/schedule/2017-2018-Serie-A-Scores-and-Fixtures"]

seriea_current = ["https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures"]

laliga = ["https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2021-2022/schedule/2021-2022-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2020-2021/schedule/2020-2021-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2019-2020/schedule/2019-2020-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2018-2019/schedule/2018-2019-La-Liga-Scores-and-Fixtures",
    "https://fbref.com/en/comps/12/2017-2018/schedule/2017-2018-La-Liga-Scores-and-Fixtures"]

laliga_current = ["https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"]

uefa_current = ["https://fbref.com/en/comps/8/schedule/Champions-League-Scores-and-Fixtures"]

uefa = ["https://fbref.com/en/comps/8/schedule/Champions-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/8/2021-2022/schedule/2021-2022-Champions-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/8/2020-2021/schedule/2020-2021-Champions-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/8/2019-2020/schedule/2019-2020-Champions-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/8/2018-2019/schedule/2018-2019-Champions-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/8/2017-2018/schedule/2017-2018-Champions-League-Scores-and-Fixtures"]

europa = ["https://fbref.com/en/comps/19/2022/schedule/2022-Scores-and-Fixtures",
        "https://fbref.com/en/comps/19/2021-2022/schedule/2021-2022-Europa-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/19/2020-2021/schedule/2020-2021-Europa-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/19/2019-2020/schedule/2019-2020-Europa-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/19/2018-2019/schedule/2018-2019-Europa-League-Scores-and-Fixtures",
        "https://fbref.com/en/comps/19/2017-2018/schedule/2017-2018-Europa-League-Scores-and-Fixtures"]


# Scrape one league at a time

sched_pages = ligue1
name = 'ligue1'

match_pages = set()
for s in sched_pages:
    html = requests.get(s).text
    match_url_regex = "\/en\/matches\/.{8}\/[^\"]+"
    matches = re.findall(match_url_regex, html)
    match_pages.update(matches)
print(f"found {len(match_pages)} matches...")

found 2236 matches...


## Mount Google drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

def parse_match(html, url):
    soup = BeautifulSoup(html, 'lxml')
    match = {}
    
    # get game meta data
    league = ' '.join(re.findall('[a-zA-Z]+', url.split('-')[-2]))
    match['match_url'] = url
    match['league_id'] = league
    match['game_id'] = url.split('/')[-2]
    match['date'], match['game_time'] = soup.select_one('span.venuetime')['data-venue-date'], soup.select_one('span.venuetime')['data-venue-time']
    
    # get team names
    title = soup.title.string.split(' Report')[0].replace('.', '').replace(' ', '_')
    title = title.split('_Match')[0]
    team_names = title.split("_vs_")
    home_team, away_team = team_names[0].replace('_', " "), team_names[1].replace('_', " ")
    match['match'], match['home_team'], match['away_team'] = title, home_team, away_team
    
    # get scores and xG
    scorebox = soup.select_one('div.scorebox')
    match['home_score'], match['away_score'] = scorebox.select_one('div.score').text.strip(), scorebox.select('div.score')[1].text.strip()
    match['home_score_xg'], match['away_score_xg'] = scorebox.select_one('div.score_xg').text.strip(), scorebox.select('div.score_xg')[1].text.strip()
    
    # get stats
    stat_table_regexes = [
        'stats_.+_passing$',
        'stats_.+_passing_types',
        'stats_.+_defense',
        'stats_.+_possession',
        'stats_.+_misc'
    ]
    for regex in stat_table_regexes:
        tables = soup.find_all('table', {'id': re.compile(regex)})
        for team_idx, table in enumerate(tables):
            team_key = 'home_' if team_idx == 0 else 'away_'
            for row in table.select('tfoot tr'):
                for cell in row.select('td'):
                    if cell.has_attr('data-stat'):
                        match[f'{team_key}{cell["data-stat"]}'] = cell.text.strip()
                        
    # get goalkeeper stats
    keeper_tables = soup.find_all('table', {'id': re.compile('keeper_stats_')})
    for team_idx, table in enumerate(keeper_tables):
        team_key = 'home_' if team_idx == 0 else 'away_'
        row = table.select('tr')[1]
        for cell in row.select('td')[3:]:
            if cell.has_attr('data-stat'):
                match[f'{team_key}{cell["data-stat"]}'] = cell.text.strip()
    
    return match

The main changes I made include:

I separated the parsing of each match into a separate function get_match_data, which takes a single match page URL as input and returns the parsed data as output. This makes it easier to parallelize the scraping process later.

I added exception handling to get_match_data so that it returns None if parsing fails for any reason.

I replaced the for loop that processes each match page with a concurrent.futures.ThreadPoolExecutor that spawns worker threads to process the match pages in parallel. This can greatly speed up the scraping process, especially when there are a large number of match pages to process.

I added a check to skip over matches that failed to parse, so that they don't get included in the final matches list. This helps ensure that the output data is as clean and accurate as possible.

To save the CSV file with the current date in the filename, you can use the strftime function from the datetime module to format the current date and time as a string. Here's an updated version of the code that saves the file with the current date and appends new matches to the existing file:

In [5]:
import os
import datetime as dt
import pandas as pd
import requests
import time

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
session.headers.update(headers)
# load the list of scraped matches from file if it exists, otherwise initialize it
scraped_matches_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.txt'
if os.path.exists(scraped_matches_file):
    with open(scraped_matches_file, 'r') as f:
        scraped_matches = f.read().splitlines()
else:
    scraped_matches = []

matches = []
urls = []
for i in match_pages:
  url = 'https://fbref.com' + i
  urls.append(url)
url_set = set(urls)
scraped_matches_set = set(scraped_matches)
to_scrape_urls = url_set.difference(scraped_matches_set)
to_scrape_urls = list(to_scrape_urls)

if len(to_scrape_urls) != 0:
  for url in to_scrape_urls:
      response = session.get(url)
      html = response.text
      
      success = False
      while not success:
          try:
              match = parse_match(html, url)
              matches.append(match)
              # append scraped matches to a list
              scraped_matches.append(url)
              # show that some activity is taking place
              if len(matches)%100==0: print(len(matches), dt.datetime.now().time())
              success = True
          except:
              time.sleep(3) # wait 3 seconds before trying again
      
      time.sleep(2) # wait 2 seconds before making another request

  # save scraped URLs to file
  with open(scraped_matches_file, 'w') as f:
      for url in scraped_matches:
          f.write(url + '\n')

  print(f'Scraped {len(matches)} matches')

  # convert list of dictionaries to DataFrame
  df = pd.DataFrame.from_dict(matches)

  # load existing CSV file if it exists
  csv_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv'
  if os.path.exists(csv_file):
      existing_df = pd.read_csv(csv_file)
      # filter out matches that have already been scraped
      existing_urls = set(existing_df['match_url'])
      new_matches = df[~df['match_url'].isin(existing_urls)]
      # append new matches to existing DataFrame
      updated_df = pd.concat([existing_df, new_matches], ignore_index=True)
  else:
      updated_df = df

  # save updated DataFrame to CSV file with current date in filename
  #date_str = dt.datetime.now().strftime('%Y-%m-%d')
  updated_df.to_csv(f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv', index=False)

  print(f'Saved updated matches to CSV file: {name}_scraped_matches.csv')
else:
  print("The matches are upto date")


Scraped 8 matches
Saved updated matches to CSV file: epl_scraped_matches.csv


In [6]:
!pip install ratelimiter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import os
import datetime as dt
import pandas as pd
import requests
import time
from ratelimiter import RateLimiter
import random

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
session.headers.update(headers)
# load the list of scraped matches from file if it exists, otherwise initialize it
scraped_matches_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.txt'
if os.path.exists(scraped_matches_file):
    with open(scraped_matches_file, 'r') as f:
        scraped_matches = f.read().splitlines()
else:
    scraped_matches = []

matches = []
urls = []
for i in match_pages:
  url = 'https://fbref.com' + i
  urls.append(url)
url_set = set(urls)
scraped_matches_set = set(scraped_matches)
to_scrape_urls = url_set.difference(scraped_matches_set)
to_scrape_urls = list(to_scrape_urls)

if len(to_scrape_urls) != 0:

  # Set the maximum number of requests per second
  rate_limiter = RateLimiter(max_calls=2, period=1)

  for url in to_scrape_urls:
      # Use the rate limiter to control the number of requests made per second
      with rate_limiter:
          response = session.get(url)
          html = response.text

      success = False
      while not success:
          try:
              match = parse_match(html, url)
              matches.append(match)
              # append scraped matches to a list
              scraped_matches.append(url)
              # show that some activity is taking place
              if len(matches) % 100 == 0:
                  print(len(matches), dt.datetime.now().time())
              success = True
          except:
              time.sleep(3)  # wait 3 seconds before trying again

      # Add a random delay between 1 and 5 seconds
      time.sleep(random.uniform(1, 5))

  # save scraped URLs to file
  with open(scraped_matches_file, 'w') as f:
      for url in scraped_matches:
          f.write(url + '\n')

  print(f'Scraped {len(matches)} matches')

  # convert list of dictionaries to DataFrame
  df = pd.DataFrame.from_dict(matches)

  # load existing CSV file if it exists
  csv_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv'
  if os.path.exists(csv_file):
      existing_df = pd.read_csv(csv_file)
      # filter out matches that have already been scraped
      existing_urls = set(existing_df['match_url'])
      new_matches = df[~df['match_url'].isin(existing_urls)]
      # append new matches to existing DataFrame
      updated_df = pd.concat([existing_df, new_matches], ignore_index=True)
  else:
      updated_df = df

  # save updated DataFrame to CSV file with current date in filename
  #date_str = dt.datetime.now().strftime('%Y-%m-%d')
  updated_df.to_csv(f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv', index=False)

  print(f'Saved updated matches to CSV file: {name}_scraped_matches.csv')
else:
  print("The matches are upto date")

KeyboardInterrupt: ignored

# Speeded Up with ChatGPT(Bing)

In [10]:
import os
import datetime as dt
import pandas as pd
import requests
import time
from ratelimiter import RateLimiter
import random
import concurrent.futures

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
session.headers.update(headers)
# load the list of scraped matches from file if it exists, otherwise initialize it
scraped_matches_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.txt'
if os.path.exists(scraped_matches_file):
    with open(scraped_matches_file, 'r') as f:
        scraped_matches = f.read().splitlines()
else:
    scraped_matches = []

matches = []
urls = []
for i in match_pages:
  url = 'https://fbref.com' + i
  urls.append(url)
url_set = set(urls)
scraped_matches_set = set(scraped_matches)
to_scrape_urls = url_set.difference(scraped_matches_set)
to_scrape_urls = list(to_scrape_urls)

if len(to_scrape_urls) != 0:

  # Set the maximum number of requests per second
  rate_limiter = RateLimiter(max_calls=2, period=1)

  def scrape_url(url):
      # Use the rate limiter to control the number of requests made per second
      with rate_limiter:
          response = session.get(url)
          html = response.text

      success = False
      while not success:
          try:
              match = parse_match(html, url)
              matches.append(match)
              # append scraped matches to a list
              scraped_matches.append(url)
              # show that some activity is taking place
              if len(matches) % 100 == 0:
                  print(len(matches), dt.datetime.now().time())
              success = True
          except:
              time.sleep(3)  # wait 3 seconds before trying again

      # Add a random delay between 1 and 5 seconds
      time.sleep(random.uniform(1, 5))

  # Use a ThreadPoolExecutor to make multiple requests at the same time
  """
  This code uses the concurrent.futures module to create a ThreadPoolExecutor 
  that can run multiple instances of the scrape_url function at the same time. 
  This can significantly reduce the time it takes to scrape a large number of pages.
  """
  with concurrent.futures.ThreadPoolExecutor() as executor:
      executor.map(scrape_url, to_scrape_urls)

  # save scraped URLs to file
  with open(scraped_matches_file, 'w') as f:
      for url in scraped_matches:
          f.write(url + '\n')

  print(f'Scraped {len(matches)} matches')

  # convert list of dictionaries to DataFrame
  df = pd.DataFrame.from_dict(matches)

  # load existing CSV file if it exists
  csv_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv'
  if os.path.exists(csv_file):
      existing_df = pd.read_csv(csv_file)
      # filter out matches that have already been scraped
      existing_urls = set(existing_df['match_url'])
      new_matches = df[~df['match_url'].isin(existing_urls)]
      # append new matches to existing DataFrame
      updated_df = pd.concat([existing_df, new_matches], ignore_index=True)
  else:
      updated_df = df

  # save updated DataFrame to CSV file with current date in filename
  #date_str = dt.datetime.now().strftime('%Y-%m-%d')
  updated_df.to_csv(f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv', index=False)

  print(f'Saved updated matches to CSV file: {name}_scraped_matches.csv')
else:
  print("The matches are upto date")

KeyboardInterrupt: ignored

In [14]:
!pip install requests_cache

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 2.8 MB/s eta 0:00:00


## Another Attempt at improving speed

In [ ]:
import os
import datetime as dt
import pandas as pd
import requests
import time
from ratelimiter import RateLimiter
import random
import concurrent.futures
import requests_cache

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
session.headers.update(headers)
# load the list of scraped matches from file if it exists, otherwise initialize it
scraped_matches_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.txt'
if os.path.exists(scraped_matches_file):
    with open(scraped_matches_file, 'r') as f:
        scraped_matches = f.read().splitlines()
else:
    scraped_matches = []

matches = []
urls = []
for i in match_pages:
  url = 'https://fbref.com' + i
  urls.append(url)
url_set = set(urls)
scraped_matches_set = set(scraped_matches)
to_scrape_urls = url_set.difference(scraped_matches_set)
to_scrape_urls = list(to_scrape_urls)

if len(to_scrape_urls) != 0:

  # Set up a cache to store responses to requests
  requests_cache.install_cache('scraper_cache')

  # Set the maximum number of requests per second
  rate_limiter = RateLimiter(max_calls=2, period=1)

  def scrape_url(url):
      # Use the rate limiter to control the number of requests made per second
      with rate_limiter:
          response = session.get(url)
          html = response.text

      success = False
      while not success:
          try:
              match = parse_match(html, url)
              matches.append(match)
              # append scraped matches to a list
              scraped_matches.append(url)
              # show that some activity is taking place
              if len(matches) % 100 == 0:
                  print(len(matches), dt.datetime.now().time())
              success = True
          except:
              time.sleep(3)  # wait 3 seconds before trying again

      # Add a random delay between 1 and 5 seconds
      time.sleep(random.uniform(1, 5))

  # Use a ThreadPoolExecutor to make multiple requests at the same time
  with concurrent.futures.ThreadPoolExecutor() as executor:
      executor.map(scrape_url, to_scrape_urls)

  # save scraped URLs to file
  with open(scraped_matches_file, 'w') as f:
      for url in scraped_matches:
          f.write(url + '\n')

  print(f'Scraped {len(matches)} matches')

  # convert list of dictionaries to DataFrame
  df = pd.DataFrame.from_dict(matches)

  # load existing CSV file if it exists
  csv_file = f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv'
  if os.path.exists(csv_file):
      existing_df = pd.read_csv(csv_file)
      # filter out matches that have already been scraped
      existing_urls = set(existing_df['match_url'])
      new_matches = df[~df['match_url'].isin(existing_urls)]
      # append new matches to existing DataFrame
      updated_df = pd.concat([existing_df, new_matches], ignore_index=True)
  else:
      updated_df = df

  # save updated DataFrame to CSV file with current date in filename
  #date_str = dt.datetime.now().strftime('%Y-%m-%d')
  updated_df.to_csv(f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv', index=False)

  print(f'Saved updated matches to CSV file: {name}_scraped_matches.csv')
else:
  print("The matches are upto date")

# Use Random time to boost speed

In [ ]:
import os
import datetime as dt
import pandas as pd
import requests
import time
import random

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
session.headers.update(headers)

# load the list of scraped matches from file if it exists, otherwise initialize it
scraped_matches_file = f'/content/drive/MyDrive/colab_data/{name}scraped_matches.txt'
if os.path.exists(scraped_matches_file):
    with open(scraped_matches_file, 'r') as f:
        scraped_matches = f.read().splitlines()
else:
    scraped_matches = []

matches = []
urls = []
for i in match_pages:
  url = 'https://fbref.com' + i
  urls.append(url)
url_set = set(urls)
scraped_matches_set = set(scraped_matches)
to_scrape_urls = url_set.difference(scraped_matches_set)
to_scrape_urls = list(to_scrape_urls)

if len(to_scrape_urls) != 0:
  for url in to_scrape_urls:
      response = session.get(url)
      html = response.text
      
      success = False
      while not success:
          try:
              match = parse_match(html, url)
              matches.append(match)
              # append scraped matches to a list
              scraped_matches.append(url)
              # show that some activity is taking place
              if len(matches)%100==0: print(len(matches), dt.datetime.now().time())
              success = True
          except:
              # wait for a random amount of time between 1 and 3 seconds before trying again
              time.sleep(random.uniform(1, 3))

      # wait for a random amount of time between 1 and 3 seconds before making another request
      time.sleep(random.uniform(1, 3))

  # save scraped URLs to file
  with open(scraped_matches_file, 'w') as f:
      for url in scraped_matches:
          f.write(url + '\n')

  print(f'Scraped {len(matches)} matches')

  # convert list of dictionaries to DataFrame
  df = pd.DataFrame.from_dict(matches)

  # load existing CSV file if it exists
  csv_file = 'scraped_matches.csv'
  if os.path.exists(csv_file):
      existing_df = pd.read_csv(csv_file)
      # filter out matches that have already been scraped
      existing_urls = set(existing_df['match_url'])
      new_matches = df[~df['match_url'].isin(existing_urls)]
      # append new matches to existing DataFrame
      updated_df = pd.concat([existing_df, new_matches], ignore_index=True)
  else:
      updated_df = df

  # save updated DataFrame to CSV file with current date in filename
  #date_str = dt.datetime.now().strftime('%Y-%m-%d')
  updated_df.to_csv(f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv', index=False)
  print(f'Saved updated matches to CSV file: {name}_scraped_matches.csv')
else:
    print("The matches are up to date")


KeyboardInterrupt: ignored

In [ ]:
dft = pd.DataFrame(matches)
dft.shape

(7, 194)

In this version of the code, the pd.read_csv function is used to load the existing CSV file into a DataFrame, and the ~ operator is used with the isin function to filter out matches that have already been scraped. The new matches are then appended to the existing DataFrame using the pd.concat function with the ignore_index parameter set to True.

The strftime function is used to format the current date and time as a string with the format '%Y-%m-%d', which is then included in the filename of the CSV file that

# An attempt to Speed the Scraping Process

In [ ]:
import os
import datetime as dt
import pandas as pd
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

session = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                  '(KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
session.headers.update(headers)

# function to scrape a single match
def scrape_match(url):
    response = session.get(url)
    html = response.text
    try:
        match = parse_match(html, url)
        return match
    except:
        return None

# load the list of scraped matches from file if it exists, otherwise initialize it
scraped_matches_file = f'/content/drive/MyDrive/colab_data/{name}scraped_matches.txt'
if os.path.exists(scraped_matches_file):
    with open(scraped_matches_file, 'r') as f:
        scraped_matches = f.read().splitlines()
else:
    scraped_matches = []

matches = []
urls = []
for i in match_pages:
  url = 'https://fbref.com' + i
  urls.append(url)
url_set = set(urls)
scraped_matches_set = set(scraped_matches)
to_scrape_urls = url_set.difference(scraped_matches_set)
to_scrape_urls = list(to_scrape_urls)

if len(to_scrape_urls) != 0:
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(scrape_match, url) for url in to_scrape_urls]

        for future in as_completed(futures):
            match = future.result()
            if match is not None:
                matches.append(match)
                scraped_matches.append(match['match_url'])
                # show that some activity is taking place
                if len(matches)%100==0: print(len(matches), dt.datetime.now().time())

    # save scraped URLs to file
    with open(scraped_matches_file, 'w') as f:
        for url in scraped_matches:
            f.write(url + '\n')

    print(f'Scraped {len(matches)} matches')

    # convert list of dictionaries to DataFrame
    df = pd.DataFrame.from_dict(matches)

    # load existing CSV file if it exists
    csv_file = 'scraped_matches.csv'
    if os.path.exists(csv_file):
        existing_df = pd.read_csv(csv_file)
        # filter out matches that have already been scraped
        existing_urls = set(existing_df['match_url'])
        new_matches = df[~df['match_url'].isin(existing_urls)]
        # append new matches to existing DataFrame
        updated_df = pd.concat([existing_df, new_matches], ignore_index=True)
    else:
        updated_df = df

    # save updated DataFrame to CSV file with current date in filename
    #date_str = dt.datetime.now().strftime('%Y-%m-%d')
    updated_df.to_csv(f'/content/drive/MyDrive/colab_data/{name}_scraped_matches.csv', index=False)
    print(f'Saved updated matches to CSV file: {name}_scraped_matches.csv')
else:
    print("The matches are up to date")

Scraped 0 matches
Saved updated matches to CSV file: ligue1_scraped_matches.csv


# Try an Optimized code by ChatGPT